# Connecting to the Prompt Hub

We can connect our application to LangSmith's Prompt Hub, which will allow us to test and iterate on our prompts within LangSmith, and pull our improvements directly into our application.

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(override=True)

True

### Pull a prompt from Prompt Hub

Pull in a prompt from Prompt Hub by pasting in the code snippet from the UI.

In [2]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
import os
from langsmith import Client
LANGSMITH_API_KEY=os.getenv("LANGSMITH_API_KEY")
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("pirate-friend",)

C:\Users\Aniket\Documents\Clg\LLM - part 2\myllm\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Let's see what we pulled - note that we did not get the model, so this is just a StructuredPrompt and not runnable.

In [3]:
prompt

StructuredPrompt(input_variables=['language', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': '-', 'lc_hub_repo': 'pirate-friend', 'lc_hub_commit_hash': '536bdc12d2263899d05dd12becf7b9adae3a9f8fa518c79c36f3f34a397a38f9'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template="You are a pirate from the 1600's , you only speak {language}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})], schema_={'title': 'answer', 'description': 'Extracts the answer', 'type': 'object', 'properties': {'answer': {'type': 'string', 'description': 'the  answer from the LLM to the user'}}, 'required': ['answer'], 'additionalProperties': False, 'strict': True}, structured_output_kwargs={})

Cool! Now let's hydrate our prompt by calling .invoke() with our inputs

In [4]:
hydrated_prompt = prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})
hydrated_prompt

ChatPromptValue(messages=[SystemMessage(content="You are a pirate from the 1600's , you only speak Spanish", additional_kwargs={}, response_metadata={}), HumanMessage(content='Are you a captain yet?', additional_kwargs={}, response_metadata={})])

And now let's pass those messages to OpenAI and see what we get back!

In [5]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQGtwswezdejZImQraVlcAbRQ3Lfw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='¡Ahoy! No, aún no soy capitán, pero estoy en el camino de convertirme en uno. La vida en el mar es dura, pero con valor y astucia, algún día seré el capitán de mi propio barco. ¿Y tú, qué buscas en el océano?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760377532, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=61, prompt_tokens=32, total_tokens=93, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

##### [Extra: LangChain Only] Pulling down the Model Configuration

We can also pull down the saved model configuration as a LangChain RunnableBinding when we use `include_model=True`. This allows us to run our prompt template directly with the saved model configuration.

In [ ]:
prompt

Test out your prompt!

In [ ]:
prompt.invoke({"question": "Are you a captain yet?", "language": "Spanish"})

### Pull down a specific commit

Pull down a specific commit from the Prompt Hub by pasting in the code snippet from the UI.

In [8]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("pirate-friend")

Run this commit!

In [9]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "What is the world like?", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQGzhdNWnpafj2GXj9R0meLtiahhS', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Ahoy, matey! In the year 2500, the world be a wild blend of advanced technology and remnants of the past. The seas are still vast, but now they be filled with floating cities and mega-structures, where folk live atop the waters, and airships sail the skies like the galleons of old. \n\nEnvironmental challenges have shaped the world; climate change has led to rising oceans, reshaping coastlines and forcing many to adapt. Many areas are now lush with bioengineered flora and fauna designed to survive in altered conditions.\n\nThe digital realm is intertwined with reality, where virtual experiences blend with the physical world. Pirates like me navigate not just the waters, but also the vast networks of cyberspace, seeking treasure in both data and gold!\n\nThe law of the sea still exists, but it’s more about who can outsmart the s

### Uploading Prompts

You can also easily update your prompts in the hub programmatically.



In [10]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client

client=Client()

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""

french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
client.push_prompt("french-rag-prompt", object=french_prompt_template)

'https://smith.langchain.com/prompts/french-rag-prompt/75567b82?organizationId=b3cf18a4-4e38-4c04-bff3-e0df17ce5621'

You can also push a prompt as a RunnableSequence of a prompt and a model. This is useful for storing the model configuration you want to use with this prompt. The provider must be supported by the LangSmith playground.

In [11]:
from langchain.prompts.chat import ChatPromptTemplate
from langsmith import Client
from langchain_openai import ChatOpenAI

client=Client()
model = ChatOpenAI(model="gpt-4o-mini")

french_prompt = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation.

Your users can only speak French, make sure you only answer your users with French.

Conversation: {conversation}
Context: {context} 
Question: {question}
Answer:"""
french_prompt_template = ChatPromptTemplate.from_template(french_prompt)
chain = french_prompt_template | model
client.push_prompt("french-runnable-sequence", object=chain)

'https://smith.langchain.com/prompts/french-runnable-sequence/f1499fb1?organizationId=b3cf18a4-4e38-4c04-bff3-e0df17ce5621'

# my tweakings


In [13]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("my_promt")

In [14]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "how to clear an enterance exam", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQH5fm7UxNGEVEeSs0xM41mjjbfqF', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Clearing an entrance exam requires a combination of effective preparation, time management, and strategic studying. Here are some steps you can follow to increase your chances of success:\n\n1. **Understand the Exam Format**: Familiarize yourself with the structure of the exam, including the types of questions, marking scheme, and time limits.\n\n2. **Gather Study Materials**: Collect textbooks, past papers, study guides, and online resources that are relevant to the exam. Ensure that you have the latest and most relevant materials.\n\n3. **Create a Study Plan**: Develop a realistic study schedule that covers all the topics you need to study. Break down your syllabus into manageable sections and allocate time for each.\n\n4. **Focus on the Basics**: Ensure you have a strong grasp of the fundamental concepts before diving into a

In [15]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("my_promt")

In [16]:
from openai import OpenAI
from langsmith.client import convert_prompt_to_openai_format

openai_client = OpenAI()

hydrated_prompt = prompt.invoke({"question": "how to make a student clear an enterance exam", "language": "English"})
# NOTE: We can use this utility from LangSmith to convert our hydrated prompt to openai format
converted_messages = convert_prompt_to_openai_format(hydrated_prompt)["messages"]

openai_client.chat.completions.create(
        model="gpt-4o-mini",
        messages=converted_messages,
    )

ChatCompletion(id='chatcmpl-CQH7bVSVNF9F9drTcveeljNOObgBs', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Helping a student prepare for an entrance exam requires a structured approach and supportive guidance. Here are some effective strategies you can employ:\n\n1. **Understand the Exam Format**:\n   - Familiarize the student with the exam structure, types of questions (multiple-choice, essay, etc.), subjects involved, and scoring methodology.\n\n2. **Conduct a Diagnostic Assessment**:\n   - Assess the student's current knowledge and skills. Identify their strengths and weaknesses in the subjects that will be tested.\n\n3. **Create a Study Plan**:\n   - Develop a tailored study schedule that includes regular study sessions, breaks, and time for review. Ensure it allocates time for each subject area based on the student's needs.\n\n4. **Gather Study Materials**:\n   - Provide access to high-quality study resources such as textbooks,